Load Dspy

In [1]:
%load_ext autoreload
%autoreload 2

import sys
import os

try: # When on google Colab, let's clone the notebook so we download the cache.
    import google.colab
    repo_path = 'dspy'
    !git -C $repo_path pull origin || git clone https://github.com/stanfordnlp/dspy $repo_path
except:
    repo_path = '.'

if repo_path not in sys.path:
    sys.path.append(repo_path)

# Set up the cache for this notebook
os.environ["DSP_NOTEBOOK_CACHEDIR"] = os.path.join(repo_path, 'cache')

import pkg_resources # Install the package if it's not installed
if not "dspy-ai" in {pkg.key for pkg in pkg_resources.working_set}:
    !pip install -U pip
    !pip install dspy-ai
    !pip install openai~=0.28.1
    # !pip install -e $repo_path

import dspy

fatal: cannot change to 'dspy': No such file or directory
Cloning into 'dspy'...
remote: Enumerating objects: 33656, done.
remote: Counting objects: 100% (1453/1453), done.
remote: Compressing objects: 100% (818/818), done.
remote: Total 33656 (delta 726), reused 1276 (delta 620), pack-reused 32203
Receiving objects: 100% (33656/33656), 29.61 MiB | 10.38 MiB/s, done.
Resolving deltas: 100% (16008/16008), done.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 16.9 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 276.3/276.3 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.2/302.2 kB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.5/328.5 kB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 28.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Define a **dataset**

In [ ]:
# train = [("sarah palin is awesome. | yes she is", "sarah palin", "positive"),
#       ("russell brand and katy perry suit each other. they 're both weird.","katy perry", "negative"),
#       ("lindsay lohan must hate herself : samantha ronson has more followers on twitter. you know you 've hit rock bottom when ...", "lindsay lohan", 'neutral'),
#       ('and he has a vision too ! mirzan mahatir is looking like steve jobs right now.', 'steve jobs','neutral'),
#       ("i love google ! as a whole with all its services","google","positive")
#       ]

# # 修改train数据集，使其包含输入`aspect`
# train = [dspy.Example(sentence=sentence, aspect=aspect, sentiment=sentiment).with_inputs('sentence', 'aspect') for sentence, aspect, sentiment in train]

# # Check each entry in the dev dataset for errors
# dev = [("my 3-year-old was amazed yesterday to find that ' real ' 10 pin bowling is nothing like it is on the wii ...", "wii", "neutral"),
#     ("listening 2 '' 3 '' by ms britney spears cant wait 4 the video on oct . 19 you guys need 2 love her new single also buy her single collection nov . 24 !", "britney spears", "positive"),
#     ("ff markieai likes animals . hates spiders . hates charlie sheen . likes cookies .", "charlie sheen", "negative"),
#     ("espn soccer net news ardiles backs maradona for world cup glory", "maradona", "positive"),
#     ("-lsb- happy birthday git * git ! -rsb- ily ! ... some reason im very tired ! but today was really fun ! :-rrb- ugh my ipod died !", "ipod", "neutral"),
#     ("hilary swank as amelia earhart is actually pretty great casting.", "hilary swank", "positive"),
#     ("video : serena williams gets x-rated for new tennis game ad ! .", "serena williams", "negative"),
#     ("lmao i know !! i dnt like harry potter he 's so english !! hehehe its raining here too smdh", "harry potter", "negative"),]
# # 修改dev数据集，使其包含输入`aspect`
# dev = [dspy.Example(sentence=sentence, aspect=aspect, sentiment=sentiment).with_inputs('sentence', 'aspect') for sentence, aspect, sentiment in dev]

In [2]:
from google.colab import drive
drive.mount('/content/drive')

url= "/content/drive/MyDrive/output/rest16/test.tsv"
import pandas as pd

# Step 3: 读取TSV文件
file_path = url
df = pd.read_csv(file_path, sep='\t')
df_first_200 = df.head(200)
df_last = df.iloc[201:]
print(df_first_200.head())
print(df_last.head())
# Assuming the original DataFrame is named df
# Extracting the 'review' and 'aspect' columns
reviews = df['review'].tolist()
aspects = df['aspect'].tolist()
sentiments = df['sentiment'].tolist()

# Combining the extracted columns into the desired format
dev = [(review, aspect, "neutral" if sentiment == 0 else ("positive" if sentiment == 1 else "negative")) for review, aspect, sentiment in zip(reviews, aspects, sentiments)]

# Converting to the desired format using dspy.Example
dev = [dspy.Example(sentence=sentence, aspect=aspect, sentiment=sentiment).with_inputs('sentence', 'aspect') for sentence, aspect, sentiment in dev]

# Print the result to verify
# for example in dev:
#     print(example)

############train dataset###########
train = [("sarah palin is awesome. | yes she is", "sarah palin", "positive"),
      ("russell brand and katy perry suit each other. they 're both weird.","katy perry", "negative"),
      ("lindsay lohan must hate herself : samantha ronson has more followers on twitter. you know you 've hit rock bottom when...", "lindsay lohan", 'neutral'),
      ('and he has a vision too ! mirzan mahatir is looking like steve jobs right now.', 'steve jobs','neutral'),
      ("i love google ! as a whole with all its services","google","positive")
      ]

#
train = [dspy.Example(sentence=sentence, aspect=aspect, sentiment=sentiment).with_inputs('sentence', 'aspect') for sentence, aspect, sentiment in train]

Mounted at /content/drive
                                        review                  aspect  \
0                    Serves really good sushi.                   sushi   
1       Not the biggest portions but adequate.                portions   
2            Green Tea creme brulee is a must!  Green Tea creme brulee   
3       – I can't say enough about this place.                   place   
4  It has great sushi and even better service.                   sushi   

   sentiment  
0          1  
1          0  
2          1  
3          1  
4          1  
                                                review       aspect  sentiment
201  i highly recommend this place to all that want...  indain food          1
202                 The lunch menu is an awesome deal!   lunch menu          1
203                          plenty of food,trust me .         food          1
204                 Fresh ingrediants and super tasty.  ingrediants          1
205                     Best food, phenomina

Define the Metric

In [3]:
def detect_predicted_label(pred):
    # 将预测结果转换为小写
    pred_sentiment = pred.sentiment.lower()

    # 初始化 predicted_label 为 "unknown"
    predicted_label = "unknown"

    # 检查预测结果中的特定字符串并分类，以第一个出现的为准
    if "ositiv" in pred_sentiment:
        predicted_label = "positive"
    elif "egativ" in pred_sentiment:
        predicted_label = "negative"
    elif "eutra" in pred_sentiment:
        predicted_label = "neutral"
    elif "ositiv" and "egativ" and "eutra" in pred_sentiment:
        predicted_label = "neutral"

    return predicted_label

In [4]:
def sentiment_classification_metric(example, pred, trace=None):
    predicted_label = detect_predicted_label(pred)

    # 检查预测结果是否与实际结果相同
    return example.sentiment.lower() == predicted_label

Create the DSPy Analogies Context Program

In [5]:
import dspy
import openai
from dspy import OpenAI
import json

# openai.api_base = "https://api.gpts.vin/v1"
# openai.api_key = "sk-Pee7ZqyWixn655GY99Ff3eB6De9a4269Bf68E89505B90d48"

openai.api_base = "https://api.xiaoai.plus/v1"
openai.api_key = "sk-PZ3pu8tJkRrYv09o2aF29002270147809784D43e8241447d"


lm = OpenAI(api_base=openai.api_base,
             api_key=openai.api_key,
             model='gpt-3.5-turbo')

dspy.settings.configure(lm=lm)

dataset_name = "rest16"  # 修改为实际的数据集名称
txt_file_url = f"/content/drive/MyDrive/DsPy/Results/{dataset_name}/2ero-shot_InContext/2ero-shot_sentiment_analysis_results.txt"
# Define the DSPy implementation of sentiment polarity analysis with an aspect
class AnalogiesSentimentPolarityAnalysis(dspy.Signature):
    """Analyze the sentiment polarity of a given sentence and its aspect."""
    context = dspy.InputField(desc="Relevant examples of the sentence.")
    sentence = dspy.InputField(desc="The sentence to analyze.")
    aspect = dspy.InputField()
    sentiment = dspy.OutputField(desc="positive, neutral, or negative.")

class AnalogiesSentimentAnalyzer(dspy.Module):
    def __init__(self):
        super().__init__()
        self.analyze_sentiment = dspy.ChainOfThought(AnalogiesSentimentPolarityAnalysis)
        self.example_counter = 1
    def forward(self, sentence, aspect):
        # Use GPT to create context introduction for the aspect
        context = self.get_analogies_context(aspect)
        print(f"Context: \n{context}")
        prediction = self.analyze_sentiment(sentence=sentence, aspect=aspect, context=context)
        print(f"Prediction: {prediction}")
        return dspy.Prediction(context=context, sentiment=prediction.sentiment)

    def get_analogies_context(self, aspect):
        prompt = (
            f"Here are three examples of Aspect-Based Sentiment Analysis:\n"
            f"Example 1:\n"
            f"Sentence: 'sarah palin is awesome. | yes she is'\n"
            f"Aspect: 'sarah palin'\n"
            f"Sentiment: positive\n\n"
            f"Example 2:\n"
            f"Sentence: 'russell brand and katy perry suit each other. they 're both weird.'\n"
            f"Aspect: 'katy perry'\n"
            f"Sentiment: negative\n\n"
            f"Example 3:\n"
            f"Sentence: 'lindsay lohan must hate herself : samantha ronson has more followers on twitter. you know you 've hit rock bottom when...'\n"
            f"Aspect: 'lindsay lohan'\n"
            f"Sentiment: neutral\n\n"
        )
        return prompt

    def analyze_sentiment(self, sentence, aspect, context):
        prompt = (
            f"{context}\n"
            f"Please reference the examples above and predict the sentiment polarity (positive, neutral, or negative) of the aspect '{aspect}' in the following sentence:"
            f"Sentence: {sentence}\n"
            f"Aspect: {aspect}\n"
        )
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": prompt}
            ],
            max_tokens=50,
            n=1,
            stop=None
        )

        context_sentiment = response.choices[0].message['content'].strip()
        return context_sentiment

    # 存储细节包括context、aspect、sentence、预测的sent
    def write_to_file(self, aspect, context, sentence, prediction):
        with open(txt_file_url, 'a') as f:
            example_number = f"example{self.example_counter:02d} :"
            f.write(f"{example_number}\n")
            f.write(f"Aspect: {aspect}\n")
            f.write(f"@Sentence: {sentence}\n")
            f.write(f"Context: {context}\n")
            f.write("----------------------------------------------------\n")
            f.write(f"@@Sentiment: {prediction.sentiment}\n")
            f.write("++++++++++++++++++++++++++++++++++++++++++++++++++++\n")
            self.example_counter += 1

Cal ACC

In [6]:
dataset_name = "rest16"  # 修改为实际的数据集名称
method_name = "In-context"  # 修改为实际的方法名称
# 初始化情感分析器
analyzer = AnalogiesSentimentAnalyzer()

# 预测情感并计算准确率
correct_predictions = 0
sentiment_comparison={}

for i, example in enumerate(dev):
    result = analyzer(example['sentence'], example['aspect'])
    analyzer.write_to_file(example['aspect'], result.context, example['sentence'], result) # 记录细则

    example_key = f"example{i+1:02d}"
    sentence = example['sentence']
    original_sentiment = example['sentiment']
    predicted_label = detect_predicted_label(result)
    sentiment_comparison[example_key] = {
        'sentence': sentence,
        'aspect': example['aspect'],
        'original_sentiment': original_sentiment,
        'predicted_sentiment': predicted_label
    }
    if predicted_label == original_sentiment:
            correct_predictions += 1
    # Save the sentiment comparison dictionary to a .json file
output_path = f'/content/drive/MyDrive/DsPy/Results/{dataset_name}/2ero-shot_InContext/sentiment_comparison.json'
with open(output_path, 'w') as f:
    json.dump(sentiment_comparison, f, indent=4)

print(f"Sentiment comparison saved to {output_path}")


accuracy = correct_predictions / len(dev)

# 输出准确率并写入文件

output_file_path = f'/content/drive/MyDrive/DsPy/Results/{dataset_name}/{dataset_name}_{method_name}_accuracy1.txt'
with open(output_file_path, 'w') as f:
    f.write(f"Correct Predictions: {correct_predictions}\n")
    f.write(f"Total Predictions: {len(dev)}\n")
    f.write(f"Accuracy: {accuracy * 100:.2f}%\n")

流式输出内容被截断，只能显示最后 5000 行内容。

Prediction: Prediction(
    rationale='produce the sentiment. We analyze the actions of the employees, which in this case is ignoring the customers. This behavior can be perceived as negative, as it indicates a lack of customer service.',
    sentiment='negative'
)
Context: 
Here are three examples of Aspect-Based Sentiment Analysis:
Example 1:
Sentence: 'sarah palin is awesome. | yes she is'
Aspect: 'sarah palin'
Sentiment: positive

Example 2:
Sentence: 'russell brand and katy perry suit each other. they 're both weird.'
Aspect: 'katy perry'
Sentiment: negative

Example 3:
Sentence: 'lindsay lohan must hate herself : samantha ronson has more followers on twitter. you know you 've hit rock bottom when...'
Aspect: 'lindsay lohan'
Sentiment: neutral


Prediction: Prediction(
    rationale="produce the sentiment. We have a request from the wife, so the staff's response will determine the sentiment. If the staff was polite and helpful, the sentiment would be po

Compile the Program

In [ ]:
from dspy.teleprompt import BootstrapFewShot

# Set up the optimizer
config = dict(max_bootstrapped_demos=4, max_labeled_demos=4)
teleprompter = BootstrapFewShot(metric=sentiment_classification_metric, **config)

# Compile the program
optimized_classifier = teleprompter.compile(AnalogiesSentimentAnalyzer(), trainset=train)

dspy/cache/compiler


  0%|          | 0/5 [00:00<?, ?it/s]

Context: 
 Sure! Here are three Aspect-Based Sentiment Analysis examples with the aspect "sarah palin" included in the sentence:

1. Sentence: "Sarah Palin's speech was inspiring and powerful, she really knows how to connect with her audience."
   Predicted sentiment polarity: positive

2. Sentence: "Many people were unimpressed with Sarah Palin's performance at the debate, criticizing her lack of depth and substance."
   Predicted sentiment polarity: negative

3. Sentence: "The media's coverage of Sarah Palin has been mostly neutral, providing both positive and negative commentary on her political career


 20%|██        | 1/5 [00:03<00:14,  3.75s/it]

Prediction: Prediction(
    rationale="produce the sentiment. In the first example, the speaker is expressing positive sentiment towards Sarah Palin, indicating that they think highly of her. In the second example, the sentiment is negative as the speaker is criticizing Sarah Palin's performance. In the third example, the sentiment is neutral as the media coverage provides a mix of positive and negative commentary.",
    sentiment='positive, negative, or neutral.'
)
Context: 
 Sure, here are three Aspect-Based Sentiment Analysis examples with the aspect 'katy perry':

1. Sentence: "I absolutely love Katy Perry's new song, it's so catchy and uplifting!"
   Predicted sentiment polarity: 'positive'

2. Sentence: "I think Katy Perry's latest performance was just okay, nothing special."
   Predicted sentiment polarity: 'neutral'

3. Sentence: "I can't stand Katy Perry's music, I find it annoying and repetitive."
   Predicted sentiment polarity: 'negative'


...

Prediction: Prediction(
    rationale="produce the sentiment. We observe that the sentence contains the aspect 'lindsay lohan' and mentions a situation where she has less Twitter followers than Samantha Ronson. This could imply that she is 'hitting rock bottom' and may not be doing well. Therefore, the sentiment polarity in this sentence is likely to be negative.",
    sentiment='negative'
)
Context: 
 Certainly! Here are three examples of Aspect-Based Sentiment Analysis with the aspect 'Steve Jobs':

1. **Sentence:** "Steve Jobs revolutionized the tech industry with his innovative ideas."
   - **Sentiment Polarity:** Positive

2. **Sentence:** "Although controversial at times, Steve Jobs' leadership inspired a new generation of entrepreneurs."
   - **Sentiment Polarity:** Positive

3. **Sentence:** "Critics argue that Steve Jobs' management style was too demanding and autocratic."
   - **Sentiment Polarity:** Negative


 80%|████████  | 4/5 [00:25<00:07,  7.51s/it]

Prediction: Prediction(
    rationale='produce the sentiment. The sentence compares Mirzan Mahatir to Steve Jobs, suggesting that he has a vision similar to the innovative and revolutionary style of Steve Jobs. This comparison implies admiration and positive sentiment towards Steve Jobs.',
    sentiment='positive'
)
Context: 
 Sure, here are three Aspect-Based Sentiment Analysis examples with the aspect 'google' included in the sentence:

Example 1:
Sentence: "I love using Google because it makes finding information so easy."
Predicted sentiment polarity of 'google': positive

Example 2:
Sentence: "I find Google's search results to be quite accurate and helpful."
Predicted sentiment polarity of 'google': positive

Example 3:
Sentence: "I have had some issues with Google's customer support in the past."
Predicted sentiment polarity of 'google': negative


100%|██████████| 5/5 [01:03<00:00, 12.77s/it]

Prediction: Prediction(
    rationale='Context:\nSure, here are three Aspect-Based Sentiment Analysis examples with the aspect \'google\' included in the sentence:\n\nExample 1:\nSentence: "I love using Google because it makes finding information so easy."\nPredicted sentiment polarity of \'google\': positive\n\nExample 2:\nSentence: "I find Google\'s search results to be quite accurate and helpful."\nPredicted sentiment polarity of \'google\': positive\n\nExample 3:\nSentence: "I have had some issues with Google\'s customer support in the past."\nPredicted sentiment polarity of \'google\': negative\n\nSentence: i love google ! as a whole with all its services\n\nAspect: google\n\nReasoning: Let\'s think step by step in order to produce the sentiment. We notice the use of the word "love," which expresses a strong positive emotion. Additionally, the phrase "as a whole with all its services" indicates a comprehensive appreciation. Therefore, the sentiment towards Google is clearly positi

Evaluate the Program

In [ ]:
from dspy.evaluate import Evaluate

# Set up the evaluator
evaluate = Evaluate(devset=dev, metric=sentiment_classification_metric, num_threads=4, display_progress=True, display_table=0)

# Evaluate the optimized classifier
evaluate(optimized_classifier)

  0%|          | 0/8 [00:00<?, ?it/s]

Context: 
 1. Sentence: "The wii console is incredibly fun and innovative, providing hours of entertainment."
   Predicted sentiment polarity: positive

2. Sentence: "I found the wii remote to be unresponsive and frustrating to use."
   Predicted sentiment polarity: negative

3. Sentence: "Overall, the wii experience was enjoyable, with a great selection of games."
   Predicted sentiment polarity: positive
Context: 
 Sure, here are three examples of Aspect-Based Sentiment Analysis with the aspect "Maradona" included in the sentence:

1. Sentence: "Maradona was a legendary football player admired by millions around the world."
   Predicted sentiment polarity: Positive

2. Sentence: "Some fans criticized Maradona for his controversial behavior off the pitch."
   Predicted sentiment polarity: Negative

3. Sentence: "The football world mourned the loss of Maradona, a true icon in the sport."
   Predicted sentiment polarity: Neutral
Context: 
 Sure, here are three Aspect-Based Sentiment Ana

Average Metric: 0 / 1  (0.0):  12%|█▎        | 1/8 [00:03<00:22,  3.27s/it]

Prediction: Prediction(
    rationale="determine the sentiment. The sentence describes a negative experience with the wii, as the perception of real bowling differs from the wii version. This implies a sense of disappointment or dissatisfaction with the wii's portrayal of the activity.",
    sentiment='negative'
)


Average Metric: 2 / 3  (66.7):  25%|██▌       | 2/8 [00:04<00:11,  1.85s/it]

...

Average Metric: 5 / 7  (71.4):  88%|████████▊ | 7/8 [00:08<00:00,  1.01it/s]

Context: 
 Sure, here are three examples of Aspect-Based Sentiment Analysis with the aspect 'serena williams':

1. Sentence: "Serena Williams played an amazing game and showed her exceptional skill on the court."
   Predicted sentiment polarity: positive

2. Sentence: "Many fans were disappointed by Serena Williams's performance in the final match."
   Predicted sentiment polarity: negative

3. Sentence: "The media coverage of Serena Williams's comeback after injury has been overwhelmingly positive."
   Predicted sentiment polarity: positive
Prediction: Prediction(
    rationale='produce the sentiment. We can see that the speaker does not like Harry Potter because they find it "so English" and also uses "lmao" and "hehehe," indicating a light-hearted and amused tone. This suggests a negative sentiment towards Harry Potter.',
    sentiment='negative'
)


Average Metric: 6 / 8  (75.0): 100%|██████████| 8/8 [00:11<00:00,  1.39s/it]

Prediction: Prediction(
    rationale='produce the sentiment. In this sentence, the aspect \'serena williams\' is mentioned in the context of a new tennis game ad. The use of "x-rated" in the sentence indicates a negative or controversial tone. The sentiment seems to be negative as it implies that the ad featuring Serena Williams is receiving criticism.',
    sentiment='negative'
)


75.0

Query the history responses of GPT

In [ ]:
lm.inspect_history(n=1)




Analyze the sentiment polarity of a given sentence and its aspect.

---

Sentence: i love google ! as a whole with all its services
Aspect: google
Sentiment: positive

Sentence: and he has a vision too ! mirzan mahatir is looking like steve jobs right now.
Aspect: steve jobs
Sentiment: neutral

Sentence: lindsay lohan must hate herself : samantha ronson has more followers on twitter. you know you 've hit rock bottom when ...
Aspect: lindsay lohan
Sentiment: neutral

Sentence: russell brand and katy perry suit each other. they 're both weird.
Aspect: katy perry
Sentiment: negative

---

Follow the following format.

Context: Relevant examples of the sentence.

Sentence: The sentence to analyze.

Aspect: ${aspect}

Reasoning: Let's think step by step in order to ${produce the sentiment}. We ...

Sentiment: positive, neutral, or negative.

---

Context:
Sure, here are three examples of Aspect-Based Sentiment Analysis with the aspect 'serena williams':

1. Sentence: "Serena Williams pla

'\n\n\nAnalyze the sentiment polarity of a given sentence and its aspect.\n\n---\n\nSentence: i love google ! as a whole with all its services\nAspect: google\nSentiment: positive\n\nSentence: and he has a vision too ! mirzan mahatir is looking like steve jobs right now.\nAspect: steve jobs\nSentiment: neutral\n\nSentence: lindsay lohan must hate herself : samantha ronson has more followers on twitter. you know you \'ve hit rock bottom when ...\nAspect: lindsay lohan\nSentiment: neutral\n\nSentence: russell brand and katy perry suit each other. they \'re both weird.\nAspect: katy perry\nSentiment: negative\n\n---\n\nFollow the following format.\n\nContext: Relevant examples of the sentence.\n\nSentence: The sentence to analyze.\n\nAspect: ${aspect}\n\nReasoning: Let\'s think step by step in order to ${produce the sentiment}. We ...\n\nSentiment: positive, neutral, or negative.\n\n---\n\nContext:\nSure, here are three examples of Aspect-Based Sentiment Analysis with the aspect \'serena 